In [1]:
import json
import pickle
import numpy as np
import pandas as pd
from utils import *

with open("data/results/buckets.pkl", "rb") as f:
    buckets = pickle.load(f)

with open("data/mol_bits.pkl", "rb") as f:
    mol_bits = pickle.load(f)

with open("data/results/test_mols_neighbors.pkl", "rb") as f:
    test_mols_neighbors = pickle.load(f)


train = pd.read_csv("data/activity_train.csv", header=None)
train.columns = ["uniprot_id", "mol_id", "activity"]
train["mol_id"] = train["mol_id"].astype(str).str.strip()

test = pd.read_csv("data/activity_test_blanked.csv", header=None)
test.columns = ["uniprot_id", "mol_id", "activity"]
test["mol_id"] = test["mol_id"].astype(str).str.strip()

train_pivot = train.pivot(
    index="uniprot_id", columns="mol_id", values="activity"
).fillna(0)

mol_ids = list(mol_bits.keys())
prot_ids = list(train["uniprot_id"].unique())

# Baseline Estimation for CF
- Define similarity $s_{ij}$ of molecules i and j.
- Select k nearest neighbors N(i;x).
    - Molecules most similar to i, that have activity on protein x.
- Estimate rating $r_{xi}$ as the weighted average:

$$ r_{xi} = b_{xi} + \frac{\sum_{j\in N(i;x)} S_{ij} \cdot (r_{xj} - b_{xj})}{\sum_{j\in N(i;x)} s_{ij}} $$
Where $b_{xi}$ is the baseline estimate for $r_{xi}$:
$$ b_{xi} = \mu + b_x + b_i $$

- $\mu$ - overall mean protein activity
- $b_x$ - activity deviation of molecule x (molecule bias)
- $b_i$ - activity deviation of protein i (protein bias)


## Obtaining $b_{xi}$

In [2]:
def Build_bxi(df):
    mu = df.mean(axis=1).mean()
    bx = df.mean(axis=0) - mu
    bi = df.mean(axis=1) - mu

    bxi = pd.DataFrame(np.full(df.shape, mu), index=df.index, columns=df.columns)
    for i, row in df.iterrows():
        bxi.loc[i] = mu + bx + bi.loc[i]
    return bxi, bx, bi

## Similarities and Nearest Neighbors

- Jaccard similarity - $J(A, B) = \frac{A \cap B}{ A \cup B}$
- Find in each bucket, the top k highest similarity neighbors.


In [159]:
# subsetting for validation

prots = 130
molecules = 67_000

validation_set = (
    train_pivot.iloc[prots:, molecules:]
    .reset_index()
    .melt(id_vars="uniprot_id", value_name="activity")
)
validation_set = validation_set[validation_set["activity"] != 0]
validation_set["activity"] = validation_set["activity"].astype(int)

# reno
training_set = (
    train.set_index(["uniprot_id", "mol_id"])
    .drop(validation_set.set_index(["uniprot_id", "mol_id"]).index)
    .reset_index()
)

training_set_pivot = training_set.pivot(
    index="uniprot_id", columns="mol_id", values="activity"
).fillna(0)

In [155]:
bxi, bx, bi = Build_bxi(training_set_pivot)
bxi

mol_id,CHEMBL10,CHEMBL1000,CHEMBL100003,CHEMBL100004,CHEMBL100045,CHEMBL100052,CHEMBL10007,CHEMBL100071,CHEMBL10009,CHEMBL100104,...,CHEMBL99939,CHEMBL99967,CHEMBL9997,CHEMBL99979,CHEMBL9998,CHEMBL99982,CHEMBL99983,CHEMBL99987,CHEMBL9999,CHEMBL99994
uniprot_id,,,,,,,,,,,,,,,,,,,,,
O14842,0.003534,-0.010355,-0.045077,-0.045077,0.114645,0.066034,0.031312,-0.045077,0.072978,0.045201,...,0.003534,0.024367,0.072978,0.003534,-0.024244,0.010478,-0.031188,0.003534,-0.045077,-0.031188
O43193,-0.000608,-0.014497,-0.049219,-0.049219,0.110503,0.061892,0.027169,-0.049219,0.068836,0.041058,...,-0.000608,0.020225,0.068836,-0.000608,-0.028386,0.006336,-0.035331,-0.000608,-0.049219,-0.035331
O43613,0.077565,0.063676,0.028953,0.028953,0.188676,0.140065,0.105342,0.028953,0.147009,0.119231,...,0.077565,0.098398,0.147009,0.077565,0.049787,0.084509,0.042842,0.077565,0.028953,0.042842
O43614,0.106533,0.092644,0.057921,0.057921,0.217644,0.169033,0.134310,0.057921,0.175977,0.148199,...,0.106533,0.127366,0.175977,0.106533,0.078755,0.113477,0.071810,0.106533,0.057921,0.071810
O95665,0.003367,-0.010522,-0.045245,-0.045245,0.114478,0.065867,0.031144,-0.045245,0.072811,0.045033,...,0.003367,0.024200,0.072811,0.003367,-0.024411,0.010311,-0.031356,0.003367,-0.045245,-0.031356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9UBY5,-0.002184,-0.016073,-0.050795,-0.050795,0.108927,0.060316,0.025593,-0.050795,0.067260,0.039482,...,-0.002184,0.018649,0.067260,-0.002184,-0.029962,0.004760,-0.036907,-0.002184,-0.050795,-0.036907
Q9UKP6,0.012767,-0.001122,-0.035844,-0.035844,0.123878,0.075267,0.040545,-0.035844,0.082211,0.054433,...,0.012767,0.033600,0.082211,0.012767,-0.015011,0.019711,-0.021955,0.012767,-0.035844,-0.021955
Q9Y5N1,0.306003,0.292114,0.257391,0.257391,0.417114,0.368503,0.333780,0.257391,0.375447,0.347669,...,0.306003,0.326836,0.375447,0.306003,0.278225,0.312947,0.271280,0.306003,0.257391,0.271280


In [156]:
masked_MSE(bxi.values, training_set_pivot.values)

0.5295739585814045

In [144]:
## get neighbors

# this is so when we search for neighbors, we exclude the test set
all_test_val_mol_ids = list(validation_set["mol_id"].unique()) + list(
    test["mol_id"].unique()
)
all_test_val_mol_ids = set(all_test_val_mol_ids)


# id = mol_ids.index("CHEMBL461537")
query_mol = mol_ids.index("CHEMBL461360")
prot = "Q9H3N8"

# neighbor selection for similar molecules to id molecule that also
# have activity with query protein

# 1. get the neighbors
# 2. check if the neighbors have activity for the protein

neighbors = k_neighbors(
    query_mol, buckets, mol_bits, mol_ids, k=5, test_ids=all_test_val_mol_ids
)

neighbor_ids = [mol_ids[neighbors[i][0]] for i in range(len(neighbors))]

sims_to_calc = []
activities_to_calc = []
for i, nei_id in enumerate(neighbor_ids):
    subset = training_set.query(f"mol_id == '{nei_id}' & uniprot_id == '{prot}'")
    if subset.shape[0] > 0:
        sims_to_calc.append(neighbors[i][1])
        activities_to_calc.append(subset["activity"].values[0])
        # print(f"Neighbor {i} has activity")
        # print(subset)
    else:
        # print(f"Neighbor {i} does not have activity")
        continue

sims_to_calc = np.array(sims_to_calc)
activities_to_calc = np.array(activities_to_calc)

r = sims_to_calc @ activities_to_calc / sims_to_calc.sum()

print(sims_to_calc)
print(activities_to_calc)

r = sims_to_calc @ activities_to_calc / sims_to_calc.sum()
print("Activity calculated", r)
print(
    "Activity real",
    validation_set.query(f"mol_id == '{mol_ids[query_mol]}' & uniprot_id == '{prot}'")[
        "activity"
    ].values[0],
)


# ## does the neighbor have activity for said interaction protein-molecule
# neighbors = [nei for nei in neighbors if mol_ids[nei[0]] in bxi.columns]
# neis_ids = [mol_ids[nei[0]] for nei in neighbors]
# neis_sims = np.array([nei[1] for nei in neighbors])
# print(neis_ids, neis_sims)


# if len(neighbors) == 0:
#     print("No neighbors found")
# else:
#     # rxj
#     similar_mols_activities = training_set[
#         (training_set["uniprot_id"] == prot) & (training_set["mol_id"].isin(neis_ids))
#     ]

#     print(similar_mols_activities)


# bxi

[0.70769231]
[1]
Activity calculated 1.0
Activity real 1


$$ r_{xi} = b_{xi} + \frac{\sum_{j\in N(i;x)} S_{ij} \cdot (r_{xj} - b_{xj})}{\sum_{j\in N(i;x)} s_{ij}} $$


In [161]:
def predict_activity(query_mol, prot, training_set, mol_ids, buckets, mol_bits):
    neighbors = k_neighbors(
        query_mol, buckets, mol_bits, mol_ids, k=5, test_ids=all_test_val_mol_ids
    )

    neighbor_ids = [mol_ids[neighbors[i][0]] for i in range(len(neighbors))]

    sims_to_calc = []
    activities_to_calc = []
    for i, nei_id in enumerate(neighbor_ids):
        subset = training_set.query(f"mol_id == '{nei_id}' & uniprot_id == '{prot}'")
        if subset.shape[0] > 0:
            sims_to_calc.append(neighbors[i][1])
            activities_to_calc.append(subset["activity"].values[0])
            # print(f"Neighbor {i} has activity")
            # print(subset)
        else:
            # print(f"Neighbor {i} does not have activity")
            continue

    if len(sims_to_calc) == 0:
        return 0
    else:
        activities_to_calc = np.array(activities_to_calc)
        sims_to_calc = np.array(sims_to_calc)
        r = sims_to_calc @ activities_to_calc / sims_to_calc.sum()
    return r.round(0)


for i, row in validation_set.iterrows():
    print(row["mol_id"], row["uniprot_id"])
    predicted = predict_activity(
        mol_ids.index(row["mol_id"]),
        row["uniprot_id"],
        training_set,
        mol_ids,
        buckets,
        mol_bits,
    )
    print("Predicted", predicted)
    print("Real", row["activity"])
    print("----")
    if i == 10:
        break

CHEMBL560993 Q9Y5Y4
Predicted 7.0
Real 7
----
CHEMBL561007 Q9H244
Predicted 6.0
Real 5
----
CHEMBL561011 Q99705
Predicted 1.0
Real 1
----
CHEMBL561028 Q9Y5N1
Predicted 3.0
Real 7
----
CHEMBL561029 Q9Y5N1
Predicted 1.0
Real 1
----
CHEMBL561085 Q9Y5N1
Predicted 5.0
Real 4
----
CHEMBL561092 Q99705
Predicted 1.0
Real 1
----
CHEMBL561132 Q9Y5Y4
Predicted 7.0
Real 9
----
CHEMBL561148 Q99705
Predicted 3.0
Real 2
----
CHEMBL561228 Q9Y5N1
Predicted 3.0
Real 4
----
CHEMBL561234 Q9Y5N1
Predicted 3.0
Real 2
----
CHEMBL561250 Q9H3N8
Predicted 6.0
Real 6
----
CHEMBL561250 Q9Y5N1
Predicted 7.0
Real 7
----
CHEMBL561279 Q9Y5N1
Predicted 9.0
Real 8
----
CHEMBL561293 Q9Y5N1
Predicted 5.0
Real 6
----
CHEMBL561324 Q9Y5N1
Predicted 9.0
Real 10
----
CHEMBL561355 Q9Y5N1
Predicted 3.0
Real 4
----
CHEMBL561406 Q9H244
Predicted 6.0
Real 6
----
CHEMBL561420 Q9Y5N1
Predicted 5.0
Real 5
----
CHEMBL561433 Q9Y5N1
Predicted 4.0
Real 2
----
CHEMBL561484 Q9H244
Predicted 5.0
Real 5
----
CHEMBL561539 Q9Y5Y4
Predicted 5.0

## collaborative filtering
- get neighbors 
- check if neighbors have activity with same protein
- do the equation for calculation of the $r_xi$